In [1]:
import pytz
import datetime
import pandas as pd
import glob
import os
import numpy as np
import sqlalchemy
import gc


In [2]:
import pymysql.cursors
engine_pymysql_db_BL = pymysql.connect(host='localhost',user='jian',
                         password='JubaPlus-2017',db='BigLots',
                         charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)

In [3]:
list_recent_output_files_by_month=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/actative_table_output/*.csv")

list_recent_output_files_by_month=sorted(list_recent_output_files_by_month, key=lambda x: os.stat(x).st_mtime)

list_recent_output_files_by_month

['/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/actative_table_output/BL_act_table_Google_id_201805.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/actative_table_output/BL_act_table_BL_id_201805.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/actative_table_output/BL_act_table_Google_id_201806.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/actative_table_output/BL_act_table_BL_id_201806.csv']

In [4]:
df_sample=pd.read_csv(list_recent_output_files_by_month[-1],nrows=5)
df_sample

,Event Time,utc_date,utc_time,User ID,Customer_Link,customer_id_hashed,Activity ID,url,search_term,session_sequence,activity_sequence
0,1527825620932038,2018-06-01,04:00:20.932038,AMsySZbOym30cn_NyLfTMQLfGEm7,XY146845zlQHOdDaHJYZCGj-uRpc856-xnre59rH2BK9VnfkE,2e5e9c1fad63b3a662df8ec425944e69fb983a3bce0fa2...,7060559,https://www.biglots.com/account/login.jsp?tour...,not defined,14,1
1,1527825621829631,2018-06-01,04:00:21.829631,AMsySZbOym30cn_NyLfTMQLfGEm7,XY146845zlQHOdDaHJYZCGj-uRpc856-xnre59rH2BK9VnfkE,2e5e9c1fad63b3a662df8ec425944e69fb983a3bce0fa2...,6966381,https://www.biglots.com/account/login.jsp?tour...,not defined,14,2
2,1527825621861618,2018-06-01,04:00:21.861618,AMsySZbOym30cn_NyLfTMQLfGEm7,XY146845zlQHOdDaHJYZCGj-uRpc856-xnre59rH2BK9VnfkE,2e5e9c1fad63b3a662df8ec425944e69fb983a3bce0fa2...,6965586,https://www.biglots.com/account/login.jsp?tour...,not defined,14,3
3,1527825629302257,2018-06-01,04:00:29.302257,AMsySZa8iZibMdW_Yx9aJADqlEuE,XY1468sVbOM8JEgJxGW4cBMPahkrJ6jMBBH-pkAHgZYApUD4s,978d4e022180736944ebf81fbcc7942dbe465e759d27af...,7060559,http://www.biglots.com/c/furniture/kitchen-din...,kitchen dining furniture,4,1
4,1527825629326537,2018-06-01,04:00:29.326537,AMsySZa8iZibMdW_Yx9aJADqlEuE,XY1468sVbOM8JEgJxGW4cBMPahkrJ6jMBBH-pkAHgZYApUD4s,978d4e022180736944ebf81fbcc7942dbe465e759d27af...,6178927,http://www.biglots.com/c/furniture/kitchen-din...,kitchen dining furniture,4,2


In [32]:
"CREATE TABLE Pred_Activity_BL_id\
(Event_Time bigint, \
utc_date Date, \
utc_time Time, \
User_ID varchar(64), \
Customer_Link varchar(64), \
customer_id_hashed varchar(64), \
Activity_ID varchar(16), \
url varchar(1024), \
search_term varchar(256), \
session_sequence int, \
activity_sequence int);"

'CREATE TABLE Pred_Activity_BL_id(Event_Time bigint, utc_date Date, utc_time Time, User_ID varchar(64), Customer_Link varchar(64), customer_id_hashed varchar(64), Activity_ID varchar(16), url varchar(1024), search_term varchar(256), session_sequence int, activity_sequence int);'

In [46]:
def create_Pred_act_table():
    with engine_pymysql_db_BL.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS Pred_Activity_BL_id")
        cur.execute("CREATE TABLE Pred_Activity_BL_id\
(Event_Time bigint, \
date_utc date, \
time_utc time, \
User_ID varchar(64), \
Customer_Link varchar(64), \
customer_id_hashed varchar(64), \
Activity_ID varchar(16), \
url varchar(1024), \
search_term varchar(256), \
session_sequence int, \
activity_sequence int\
);")
    print("An empty TABLE Pred_Activity_BL_id has been created.",datetime.datetime.now())
 
 
create_Pred_act_table()


An empty TABLE Pred_Activity_BL_id has been created. 2020-03-06 16:32:01.476483


In [4]:
import paramiko

host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

remote_mapping_list=sftp.listdir("/mnt/drv5/lr_biglots_data/download_logs/others/")
remote_mapping_list.sort()


In [5]:
remote_mapping_list_1=[x for x in remote_mapping_list if "allrewards" in x]
remote_mapping_list_1_0908=[x for x in remote_mapping_list_1 if "0908" in x]
remote_mapping_list_1_0908=remote_mapping_list_1_0908[-1]

remote_mapping_list_1_1011=[x for x in remote_mapping_list_1 if "1011" in x]
remote_mapping_list_1_1011=remote_mapping_list_1_1011[-1]

remote_mapping_list_1=[remote_mapping_list_1_0908,remote_mapping_list_1_1011]
remote_mapping_list_1

['CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz',
 'CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz']

In [6]:
remote_mapping_list_2=[x for x in remote_mapping_list if "mapping" in x]
remote_mapping_list_2.sort()
remote_mapping_list_2

['CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190204_140255_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q2_JL_2019-09-23_20190926_084457_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_020719_0000.psv.gz']

In [7]:
remote_mapping_list_2.pop(0)
remote_mapping_list_2

['CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q2_JL_2019-09-23_20190926_084457_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz',
 'CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_020719_0000.psv.gz']

In [11]:
remote_mapping_list=remote_mapping_list_1+remote_mapping_list_2
local_mapping_existing_list=[os.path.basename(y) for y in glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/*.gz")]
remote_mapping_list=[x for x in remote_mapping_list if x not in local_mapping_existing_list]
remote_mapping_list

[]

In [12]:
for file in remote_mapping_list:
    remote_path="/mnt/drv5/lr_biglots_data/download_logs/others/"+file
    local_path="/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/"+file
    sftp.get(remote_path,local_path)
sftp.close()
transport.close()

In [13]:
list_mapping_files=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/*.psv.gz")
list_mapping_files=sorted(list_mapping_files,key = lambda x: x.split("_")[-3])
list_mapping_files

['/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_BL_mapping_file_2019Q2_JL_2019-09-23_20190926_084457_0000.psv.gz',
 '/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/save_dcm_lr_to_mysql/CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_00

In [16]:
file_name=list_mapping_files[-1]
date_LR_mapping=file_name.split("_")[-3]
mapping_file_example=pd.read_table(file_name,usecols=['Customer_Link','customer_id_hashed'],
                                   dtype=str,compression="gzip",sep="|",nrows=10)
mapping_file_example['LR_process_dt']=date_LR_mapping
mapping_file_example

,Customer_Link,customer_id_hashed,LR_process_dt
0,Xi1468m9iYFGKgGeCRhWcdWCyNZsIR18VF-wP_xJ2wSVlY...,cc11871d877005c4f9c523c98309cb4e42a8f226083035...,20200215
1,XY1468iHnEGasntCxCWbmzgEEbO_D3LI_VL6XpnfcBUOe0kQg,20384b436f3da73e612ef10932abb9bd501c7204fdf76e...,20200215
2,Xi1468_JAYR5SDcGGls7DvFAxJ3kKNoCpOdaRAannXs7qz...,7c3f175095ed30d0578b5fb6e6515c32af5ea055764fb8...,20200215
3,XY1468erJgqIXiLEQV3cyIe6FD-SIqy_G0rdR-aqP9n317GvM,53fd40eae0dabe86ac394b48debcfeb3bac7e7329ca5e3...,20200215
4,Xi1468gIIEI1CoSAZcy_L9MbQEZ0Tr66uK2xaPOaNICHiL...,ad85a2a983ea55dbcde74dbde9a0916e01ee085f23949c...,20200215
5,Xi1468HMAW2JpoL_UIW6sirUEaAi_y5b8B-6R72XT9c5OZ...,e64ed5cc108dd1683f1bd7b5703687ddb87425b59769db...,20200215
6,Xi1468EQTBiJ4PQzUs18XwgSQH40oPsEnz--81qfDh1Q8N...,d47547504b29fd13436a34d94f65a06fc61afbc47809e8...,20200215
7,Xi1468H6ZLEgY3PqrGF1RESi8z4EgjmEcfWca7JVzwV8gR...,2fff19098193deaa267230b8f09945eb658a5f025df2ba...,20200215
8,XY14685MKtG4IOV2YN4uNjiR4E2AJxGv7m0E7d0KyWTXf3gnA,8728b490d85cd56157c878f8594b5654f585f8945266cd...,20200215
9,XY14681vPEiSTsff9752SQ0H0G0KUtBa2ZcKxWhkB6fBkwJ3o,c023b87db9cef7bf045c958156101b6d72c892a655a448...,20200215


In [19]:
mapping_file_uid_idl=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/BL_GoogleID_IDL_mapping_20180524_20191231_JL_2020-02-23.csv",
                                nrows=10,dtype=str)
mapping_file_uid_idl=mapping_file_uid_idl.rename(columns={"file_date":"valid_since_dt"})
mapping_file_uid_idl

,Customer_Link,User ID,valid_since_dt
0,XY1468hPv3C6LUplcc9AOQzxWzY1udaza4Ln32X1741yLNDek,AMsySZYwlmbsf4rsAWEdAeTInPJe,20180524
1,Xi1468eM-c0d-GBhe0kva9Sv6cGyyAH_7WvdJSUyiRQJUq...,AMsySZbGHLhJxKnEGwG4yrTXarSO,20180524
2,XY1468dnvwCzHXgvKJd09Ngnlx8UNDKg6CY0iJTYcx04u4cG4,AMsySZYUiAwu0NWGt8xW144UK2rq,20180524
3,XY1468K_-yH0zRPT3BJGQ7Y4j1BeR26cw_sYp5F63Z-r91lJY,AMsySZYA7KfqPfG73RFmixBa8neT,20180524
4,XY1468imZ25GgmQX3BTpn-WhpwkGnSX5QAKe1kAq4u-irxwbY,AMsySZZRT9guWPj0Iyq0B072p7xl,20180524
5,XY14684MMl9cxiPgKPRvbSGFs9IMcOF3UjFSvSu3fUTbG4HNE,AMsySZaKM8uXVJnhvrnM6jDH0eXG,20180524
6,Xi1468dE2qxTBc7YNegn-DozvMjVpFfKngNpMOxpB5CDjC...,AMsySZbzc6gEYYuyi7KNZ6Em0oet,20180524
7,XY1468gCd3rnL4UjpZI4sPasXH7CPo0WWjgT5_bl8MaMKBeI4,AMsySZYHYxkJMnXFthSJvlD-Lpbf,20180524
8,XY1468uQ74MVSekIUaPBpNenXbOVSzR5-MSKS6qnvfgqj0ohU,AMsySZZJ6kuvdMmqO1x7nANgjO4m,20180524
9,XY1468GqbryDvXrk1qC4kNq6IgwJGRmH2fkhrhY0VALlGKMCA,AMsySZbX_CpTfTpNI7joXVQC0Vpy,20180524


In [22]:
pd.options.display.max_columns=111
list_lr_returned_impr=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/impressions/*.gz")
dcm_lr_returned_sample=pd.read_csv(list_lr_returned_impr[-1],
                                   nrows=10,sep="\t",compression="gzip")
dcm_lr_returned_sample

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Rendering ID,Creative Version,Site ID (DCM),Placement ID,Country Code,State/Region,Browser/Platform ID,Browser/Platform Version,Operating System ID,Designated Market Area (DMA) ID,City ID,ZIP/Postal Code,U Value,Event Type,Event Sub-Type,DBM Auction ID,DBM Request Time,DBM Advertiser ID,DBM Insertion Order ID,DBM Line Item ID,DBM Creative ID,DBM Bid Price (USD),DBM Bid Price (Partner Currency),DBM Bid Price (Advertiser Currency),DBM URL,DBM Site ID,DBM Language,DBM Adx Page Categories,DBM Matching Targeted Keywords,DBM Exchange ID,DBM Attributed Inventory Source External ID,DBM Attributed Inventory Source Is Public,DBM Ad Position,DBM Country Code,DBM Designated Market Area (DMA) ID,DBM ZIP/Postal Code,DBM State/Region ID,DBM City ID,DBM Operating System ID,DBM Browser/Platform ID,DBM Browser Timezone Offset Minutes,DBM Net Speed,DBM Matching Targeted Segments,DBM ISP ID,DBM Device Type,DBM Mobile Make ID,DBM Mobile Model ID,DBM View State,Impression ID,DBM Media Cost (USD),DBM Media Cost (Partner Currency),DBM Media Cost (Advertiser Currency),DBM Revenue (USD),DBM Revenue (Partner Currency),DBM Revenue (Advertiser Currency),DBM Total Media Cost (USD),DBM Total Media Cost (Partner Currency),DBM Total Media Cost (Advertiser Currency),DBM CPM Fee 1 (USD),DBM CPM Fee 1 (Partner Currency),DBM CPM Fee 1 (Advertiser Currency),DBM CPM Fee 2 (USD),DBM CPM Fee 2 (Partner Currency),DBM CPM Fee 2 (Advertiser Currency),DBM CPM Fee 3 (USD),DBM CPM Fee 3 (Partner Currency),DBM CPM Fee 3 (Advertiser Currency),DBM CPM Fee 4 (USD),DBM CPM Fee 4 (Partner Currency),DBM CPM Fee 4 (Advertiser Currency),DBM CPM Fee 5 (USD),DBM CPM Fee 5 (Partner Currency),DBM CPM Fee 5 (Advertiser Currency),DBM Media Fee 1 (USD),DBM Media Fee 1 (Partner Currency),DBM Media Fee 1 (Advertiser Currency),DBM Media Fee 2 (USD),DBM Media Fee 2 (Partner Currency),DBM Media Fee 2 (Advertiser Currency),DBM Media Fee 3 (USD),DBM Media Fee 3 (Partner Currency),DBM Media Fee 3 (Advertiser Currency),DBM Media Fee 4 (USD),DBM Media Fee 4 (Partner Currency),DBM Media Fee 4 (Advertiser Currency),DBM Media Fee 5 (USD),DBM Media Fee 5 (Partner Currency),DBM Media Fee 5 (Advertiser Currency),DBM Data Fees (USD),DBM Data Fees (Partner Currency),DBM Data Fees (Advertiser Currency),DBM Billable Cost (USD),DBM Billable Cost (Partner Currency),DBM Billable Cost (Advertiser Currency),Active View: Eligible Impressions,Active View: Measurable Impressions,Active View: Viewable Impressions
0,UNMATCHED,1577821333876261,8095847,23309569,455110105,126807492,1,2479413,259258156,US,NaN,30,0.0,501012,NaN,NaN,NaN,NaN,VIEW,VIEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t75bfrACZNvGLAg6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,UNMATCHED,1577822359257360,8095847,23309569,455110105,126807492,1,2479413,263277602,US,OR,30,0.0,501012,202.0,25201.0,NaN,NaN,VIEW,VIEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YxzvsSWOwTkUauhZ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Xi1468YkzP1Z-gbdXhnJS_RCDgl7SczSwxdgeJdSVCyyIy...,1577821997322670,8095847,23309569,455031659,126806694,1,2479413,259258360,US,TX,28,0.0,501013,99.0,13248.0,NaN,NaN,VIEW,VIEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAGhaOTdmwHhT3Ll,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,UNMATCHED,1577819259472017,8095847,23309569,455110105,126807492,1,2479413,263592148,US,NaN,30,0.0,501012,NaN,NaN,NaN,NaN,VIEW,VIEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20xiiNUMJPFAMNCD,0,0,0,0,0,

In [26]:
list_dcm_logs_impr=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/DCM_raw_logs_BL/impressions/*.tsv")
dcm_raw_logs_sample=pd.read_csv(list_dcm_logs_impr[-1],
                                   nrows=10,sep="\t")
dcm_raw_logs_sample

,Event Time,User ID,Advertiser ID,Campaign ID,Ad ID,Rendering ID,Creative Version,Site ID (DCM),Placement ID,Country Code,State/Region,Browser/Platform ID,Browser/Platform Version,Operating System ID,Designated Market Area (DMA) ID,City ID,ZIP/Postal Code,U Value,Event Type,Event Sub-Type,Impression ID,Partner1 ID
0,1577751654219776,AMsySZbVr83Ozjc7ekTFpQzepnjh,8095847,23309569,454945974,123607798,1,5174983,259319736,US,OH,28,0.0,501032,38,13854.0,44512,NaN,VIEW,VIEW,7E390RTUaUd6qqct,CAESEMpwTx8cjvuujxFlsDF-L0o
1,1577751688085504,AMsySZbAtyMz2SYg_YWNx_x0QCiQ,8095847,23309569,459015549,126601277,1,5577218,263935768,US,CA,0,0.0,501009,209,20709.0,93630,NaN,VIEW,VIEW,9hEBiIK9pesNe7ak,CAESELV6LBXa_5LUZ7_hKUJkgBU
2,1577752212455424,AMsySZZxy9Xh7WYzr16l4FMU0ZYQ,8095847,23309569,459015549,126601277,1,5577218,263935768,US,NY,0,0.0,501009,34,28347.0,13339,NaN,VIEW,VIEW,_jFvi4jRuhmgKLI4,CAESEAkA23ZJRiwxtB5mFXa3orw
3,1577752183128064,AMsySZZL7Kka_cS6l5oEdkUNPgEQ,8095847,23309569,459015549,126601277,1,5577218,263935768,US,MA,0,0.0,501009,8,17228.0,2360,NaN,VIEW,VIEW,WkSm3qhnKYWaLlJt,CAESEI_-vXB0wA63X73PjQpRLUE
4,1577753290653696,AMsySZbfioYWNWL3J5TNhWsDNmEo,8095847,23309569,454945974,123607798,1,5174983,259319736,US,AZ,28,0.0,22,189,13541.0,85701,NaN,VIEW,VIEW,rlxOoisTbJwHvtBd,CAESEDmM-GlZWS_hPFylpEBo6_Q
5,1577751621435392,AMsySZasmuK-cuugfB3GPBgW1vq-,8095847,23309569,454945974,123607798,1,5174983,259258183,US,OH,30,0.0,501012,49,15836.0,44883,NaN,VIEW,VIEW,jWr5vCL2zUpv1Rav,CAESEEI4CdppZSS_5KoiJJSwdnI
6,1577752707792896,AMsySZYmURye4WkDETt1Rza1baKg,8095847,23309569,454945974,123607798,1,5174983,259319736,US,NY,30,0.0,501012,3,17528.0,12589,NaN,VIEW,VIEW,Vbl80DcpstSkU8ZP,CAESEDoyjCSiLfBB0xelj__WzgU
7,1577753011994624,AMsySZalWLH1w8rXrDL3rmZnshlR,8095847,23309569,454945974,123607798,1,5174983,259258183,US,FL,28,0.0,501032,63,17270.0,32003,NaN,VIEW,VIEW,v4hExK1w4x0nW8Pq,CAESEHrxf5FLWDRz8k88z8buCH0
8,1577752012062720,AMsySZbVEOZ-zYSsy_U8YnTIU7Rt,8095847,23309569,459015549,126601277,1,5577218,263935768,US,FL,0,0.0,501009,36,13989.0,32137,NaN,VIEW,VIEW,zVIIwPxprxUYj39S,CAESEPOIdkIgX1XmmoDjzMPVKA0
9,1577752565743616,AMsySZY_X0dNp6GgtNkoliPIbKUN,8095847,23309569,454945974,123607798,1,5174983,259319736,US,NJ,28,0.0,501013,3,NaN,8757,NaN,VIEW,VIEW,kiFBfWVZo9rdVP6R,CAESECLFkGgEObCGdI6jAkekJXA


In [28]:
df_act_lr_returned=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/LR_returned_logs_BL/activities/CL_BigLots_dcm_account7252_activity_20180526_20180527_050818_738025211_20180527_072236_0000.tsv.gz",
                              nrows=10,sep="\t")
df_act_lr_returned['Other Data'].tolist()

['u2=undefined;~oref=https://www.mohegansuncasino.com/',
 'dc_pre=CN3g1ueNo9sCFQusyAodPd4Kjw;u3=fight;u2=fight-night-liverpool-2018;u1=event;~oref=http://www.ufc.ca/event/fight-night-liverpool-2018',
 'u1=http://www.biglots.com/page/grandopening/?zcp=soc_facebook_fairhavensweeps;u2=page:grandopening;u3=contentmain;~oref=http://www.biglots.com/page/grandopening/',
 'dc_pre=CN-06Ofgo9sCFcW_wAod4SQDRQ;u1=http://www.biglots.com/?zcp=pd_srch_go&brand+-+core_est2997&gclid=cjwkcajwi6tybrayeiwaoeh7gyqsxju0z71i04yutsbaul5aobtuwgo_1otrdximw81k9ujxyv9bhhocqssqavd_bwe;u2=home;u3=home;~oref=http://www.biglots.com/',
 'dc_pre=CIWyiMq7o9sCFZZ2YgodRKoFkQ;u1=home;~oref=http://www.ufc.com/',
 'u1=http://www.biglots.com/;~oref=http://www.biglots.com/',
 'u1=http://www.biglots.com/c/outdoor/patio-furniture/medium-seats-up-to-4/_/n-3129013054/?scm=web_outdoorpartycp_6grid_all_outdoor_patiofurn_mediumspace_4l2_05202018;u2=category:outdoor:patio furniture:medium: seats up to 4;u3=category;~oref=http://www.bi

In [29]:
df_act_lr_returned

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Rendering ID,Creative Version,Site ID (DCM),Placement ID,Country Code,State/Region,Browser/Platform ID,Browser/Platform Version,Operating System ID,U Value,Activity ID,TRAN Value,Other Data,ORD Value,Interaction Time,Conversion ID,Segment Value 1,Floodlight Configuration,Event Type,Event Sub-Type,DBM Auction ID,DBM Request Time,DBM Advertiser ID,DBM Insertion Order ID,DBM Line Item ID,DBM Creative ID,DBM Bid Price (USD),DBM Bid Price (Partner Currency),DBM Bid Price (Advertiser Currency),DBM URL,DBM Site ID,DBM Language,DBM Adx Page Categories,DBM Matching Targeted Keywords,DBM Exchange ID,DBM Attributed Inventory Source External ID,DBM Attributed Inventory Source Is Public,DBM Ad Position,DBM Country Code,DBM Designated Market Area (DMA) ID,DBM ZIP/Postal Code,DBM State/Region ID,DBM City ID,DBM Operating System ID,DBM Browser/Platform ID,DBM Browser Timezone Offset Minutes,DBM Net Speed,DBM Matching Targeted Segments,DBM ISP ID,DBM Device Type,DBM Mobile Make ID,DBM Mobile Model ID,Total Conversions,Total Revenue,DBM Media Cost (USD),DBM Media Cost (Partner Currency),DBM Media Cost (Advertiser Currency),DBM Revenue (USD),DBM Revenue (Partner Currency),DBM Revenue (Advertiser Currency),DBM Total Media Cost (USD),DBM Total Media Cost (Partner Currency),DBM Total Media Cost (Advertiser Currency),DBM CPM Fee 1 (USD),DBM CPM Fee 1 (Partner Currency),DBM CPM Fee 1 (Advertiser Currency),DBM CPM Fee 2 (USD),DBM CPM Fee 2 (Partner Currency),DBM CPM Fee 2 (Advertiser Currency),DBM CPM Fee 3 (USD),DBM CPM Fee 3 (Partner Currency),DBM CPM Fee 3 (Advertiser Currency),DBM CPM Fee 4 (USD),DBM CPM Fee 4 (Partner Currency),DBM CPM Fee 4 (Advertiser Currency),DBM CPM Fee 5 (USD),DBM CPM Fee 5 (Partner Currency),DBM CPM Fee 5 (Advertiser Currency),DBM Media Fee 1 (USD),DBM Media Fee 1 (Partner Currency),DBM Media Fee 1 (Advertiser Currency),DBM Media Fee 2 (USD),DBM Media Fee 2 (Partner Currency),DBM Media Fee 2 (Advertiser Currency),DBM Media Fee 3 (USD),DBM Media Fee 3 (Partner Currency),DBM Media Fee 3 (Advertiser Currency),DBM Media Fee 4 (USD),DBM Media Fee 4 (Partner Currency),DBM Media Fee 4 (Advertiser Currency),DBM Media Fee 5 (USD),DBM Media Fee 5 (Partner Currency),DBM Media Fee 5 (Advertiser Currency),DBM Data Fees (USD),DBM Data Fees (Partner Currency),DBM Data Fees (Advertiser Currency),DBM Billable Cost (USD),DBM Billable Cost (Partner Currency),DBM Billable Cost (Advertiser Currency)
0,UNMATCHED,1527313727562624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,NJ,28,0.0,501013,NaN,2598430,NaN,u2=undefined;~oref=https://www.mohegansuncasin...,8.140530e+12,NaN,0,NaN,4635636,CONVERSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,XY1468MOucvf5CjDkiRjyElHVQ2Cy6_IOfegL-Dk3mb6hmHJM,1527328437360706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA,NF,28,0.0,22,NaN,1172217,NaN,dc_pre=CN3g1ueNo9sCFQusyAodPd4Kjw;u3=fight;u2=...,9.479938e+11,NaN,0,NaN,3941025,CONVERSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Xi1468yFMSjTW75NlyA9eaB6i5B2EIzDJOaE8lAl0BROCy...,1527332986260298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,MA,30,0.0,501012,NaN,7060559,NaN,u1=http://www.biglots.com/page/grandopening/?z...,2.977546e+12,NaN,0,NaN,8095847,CONVERSION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,XY1468AV-OZq_yFQ_-7vkHaT48Fx4sw6oT7nX9iRP2goS2rFs,1527350717775279,8095847.0,20836313.0,414888750.0,0.0,0.0,1234535.0,215929001.0,US,TX,28,0.0,501032,NaN,7060559,NaN,dc_pre=CN-06Ofgo9sCFcW_wAod

In [14]:
df_all_mapping=pd.DataFrame()
for file in list_mapping_files:
    date_LR_mapping=file.split("_")[-3]
    print(os.path.basename(file),date_LR_mapping)
    df=pd.read_table(file,dtype=str,compression="gzip",sep="|",usecols=['Customer_Link','customer_id_hashed'])
    df['valid_date_up_to']=date_LR_mapping
    print(df.columns.tolist())
    df_all_mapping=df_all_mapping.append(df)
print(df_all_mapping.shape)


CL_BigLots_allrewards_combined0908_20181029_171317_0000.psv.gz 20181029
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_allrewards_combined1011_20181026_232208_0000.psv.gz 20181026
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz 20190207
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz 20190516
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_BL_mapping_file_2019Q2_JL_2019-09-23_20190926_084457_0000.psv.gz 20190926
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_BL_mapping_file_2019Q3_JL_2019-11-20_20191125_232935_0000.psv.gz 20191125
['Customer_Link', 'customer_id_hashed', 'segment']
CL_BigLots_BL_mapping_file_2019Q4_JL_2020-02-14_20200215_020719_0000.psv.gz 20200215
['Customer_Link', 'customer_id_hashed', 'segment']


In [18]:
df_gid_idl=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/UID_IDL_mapping/BL_GoogleID_IDL_mapping_20180524_20191231_JL_2020-02-23.csv",
                      dtype=str)
df_gid_idl.head(3)

,Customer_Link,User ID,file_date
0,XY1468hPv3C6LUplcc9AOQzxWzY1udaza4Ln32X1741yLNDek,AMsySZYwlmbsf4rsAWEdAeTInPJe,20180524
1,Xi1468eM-c0d-GBhe0kva9Sv6cGyyAH_7WvdJSUyiRQJUq...,AMsySZbGHLhJxKnEGwG4yrTXarSO,20180524
2,XY1468dnvwCzHXgvKJd09Ngnlx8UNDKg6CY0iJTYcx04u4cG4,AMsySZYUiAwu0NWGt8xW144UK2rq,20180524


In [19]:
print(df_gid_idl.shape)

(30654338, 3)
